# Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid", {'grid.linestyle': '--'})
sns.set_style({'font.family':'serif', 'font.serif':'Computer Modern'})
sns.set_context(font_scale=2, rc={"font.size":10,"axes.titlesize":20,"axes.labelsize":15})
from collections import defaultdict
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

In [ ]:
!pip install xgboost

In [3]:
!pip install lightgbm

^C


In [4]:
!pip install catboost

^C


In [15]:
from xgboost import XGBClassifier

In [16]:
from lightgbm import LGBMClassifier

In [17]:
from scipy.stats import randint

In [18]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [19]:
df_train = pd.read_csv("C:/Users/Utente/OneDrive/Desktop/UNIPI - DS for Business Informatics/Data Mining_2/Progetto/df_train_processed.csv")
df_test = pd.read_csv("C:/Users/Utente/OneDrive/Desktop/UNIPI - DS for Business Informatics/Data Mining_2/Progetto/df_test_processed.csv")

In [7]:
pd.set_option('display.max_columns', None)

# Standardization

## Train

In [20]:
df_train_num = df_train.select_dtypes(include="number").drop("actor", axis=1)
df_train_cat = df_train.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_train_num_scaled = pd.DataFrame(scaler.fit_transform(df_train_num), columns=df_train_num.columns)
df_train_scaled_s = pd.concat([df_train_num_scaled, df_train_cat], axis=1)



## Test

In [21]:
df_test_num = df_test.select_dtypes(include="number").drop("actor", axis=1)
df_test_cat = df_test.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_test_num_scaled = pd.DataFrame(scaler.fit_transform(df_test_num), columns=df_test_num.columns)
df_test_scaled_s = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

# LGBM Classifier()

## target --> sex

In [22]:
### One-hot encoding

#### Train 

df_train_scaled_s['sex'] = pd.factorize(df_train_scaled_s['sex'])[0]
df_train_s = pd.get_dummies(df_train_scaled_s)

#### Test

df_test_scaled_s['sex'] = pd.factorize(df_test_scaled_s['sex'])[0]
df_test_s = pd.get_dummies(df_test_scaled_s)

In [23]:
X_train = df_train_s.drop(['sex'], axis=1)
y_train = df_train_s['sex']
X_test = df_test_s.drop(['sex'], axis=1)
y_test = df_test_s['sex']

In [52]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50), #[0.001, 0.05, 0.1, 0.2,0.5, 0.7, ],
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


[LightGBM] [Warning] subsample_for_bin is set=200000, subsample_for_bin=224482 will be ignored. Current value: subsample_for_bin=200000
Best parameters: {'boosting_type': 'goss', 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 428, 'num_leaves': 9}
              precision    recall  f1-score   support

           0       0.80      0.99      0.88       312
           1       0.99      0.75      0.85       312

    accuracy                           0.87       624
   macro avg       0.89      0.87      0.87       624
weighted avg       0.89      0.87      0.87       624



### Seconda grid

In [24]:
param_distribs = {
    'boosting_type': ['goss'],
    'n_estimators': [348,358,368,378,388,398,408,418,428,438,448,458,468,478,488,498],
    'max_depth': [3,4,5,6,7,8,9,10,11],
    'learning_rate': [0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17],
    'num_leaves': [6,7,8,9,10,11,12,13,14,15,16,17],
    "subsample_for_bin" : [200000,210000,220000,230000,240000,250000,260000]
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'subsample_for_bin': 240000, 'num_leaves': 14, 'n_estimators': 348, 'max_depth': 4, 'learning_rate': 0.16, 'boosting_type': 'goss'}
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       312
           1       0.99      0.77      0.87       312

    accuracy                           0.88       624
   macro avg       0.90      0.88      0.88       624
weighted avg       0.90      0.88      0.88       624



## Vocal_channel

In [27]:
df_test_scaled_v = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [28]:
df_train_scaled_v = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [29]:
### One-hot encoding

#### Train 

df_train_scaled_v['vocal_channel'] = pd.factorize(df_train_scaled_v['vocal_channel'])[0]
df_train_v = pd.get_dummies(df_train_scaled_v)

#### Test

df_test_scaled_v['vocal_channel'] = pd.factorize(df_test_scaled_v['vocal_channel'])[0]
df_test_v = pd.get_dummies(df_test_scaled_v)

In [30]:
X_train = df_train_v.drop(['vocal_channel'], axis=1)
y_train = df_train_v['vocal_channel']
X_test = df_test_v.drop(['vocal_channel'], axis=1)
y_test = df_test_v['vocal_channel']

In [22]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'goss', 'learning_rate': 0.32374575428176433, 'max_depth': 10, 'n_estimators': 480, 'num_leaves': 39, 'subsample_for_bin': 280077}
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       360
           1       0.95      0.99      0.97       264

    accuracy                           0.97       624
   macro avg       0.97      0.98      0.97       624
weighted avg       0.97      0.97      0.97       624



### Seconda grid

In [31]:
param_distribs = {
    'boosting_type': ['goss'],
    'n_estimators': [400,410,420,430,440,450,460,470,480,490,500,510,520,530],
    'max_depth': [4,5,6,7,8,9,10,11,12,13,14,15],
    'learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'num_leaves': [35,36,37,38,39,40,41,42,43,44,45],
    "subsample_for_bin" : [250077,260077,270077,280077,290077,300077] 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))

Best parameters: {'subsample_for_bin': 290077, 'num_leaves': 41, 'n_estimators': 450, 'max_depth': 15, 'learning_rate': 0.5, 'boosting_type': 'goss'}
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       360
           1       0.96      0.99      0.98       264

    accuracy                           0.98       624
   macro avg       0.98      0.98      0.98       624
weighted avg       0.98      0.98      0.98       624



## Emotional_intensity

In [64]:
df_test_scaled_ei = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [65]:
df_train_scaled_ei = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [66]:
### One-hot encoding

#### Train 

df_train_scaled_ei['emotional_intensity'] = pd.factorize(df_train_scaled_ei['emotional_intensity'])[0]
df_train_ei = pd.get_dummies(df_train_scaled_ei)

#### Test

df_test_scaled_ei['emotional_intensity'] = pd.factorize(df_test_scaled_ei['emotional_intensity'])[0]
df_test_ei = pd.get_dummies(df_test_scaled_ei)

In [67]:
X_train = df_train_ei.drop(['emotional_intensity'], axis=1)
y_train = df_train_ei['emotional_intensity']
X_test = df_test_ei.drop(['emotional_intensity'], axis=1)
y_test = df_test_ei['emotional_intensity']

In [68]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'dart', 'learning_rate': 0.32374575428176433, 'max_depth': 28, 'n_estimators': 209, 'num_leaves': 10, 'subsample_for_bin': 287455}
              precision    recall  f1-score   support

           0       0.86      0.68      0.76       336
           1       0.70      0.88      0.78       288

    accuracy                           0.77       624
   macro avg       0.78      0.78      0.77       624
weighted avg       0.79      0.77      0.77       624



### Seconda Grid

In [69]:
param_distribs = {
    'boosting_type': ['dart'],
    'n_estimators': [179,189,199,209,219,229,239,249,259,269],
    'max_depth': [25,26,27,28,28,30,31,32,33,34,35,36,37,38,39],
    'learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'num_leaves': [5,6,7,8,9,10,11,12,13,14,15],
    "subsample_for_bin" : [257455,267455,277455,287455,297455,307455,317455,327455,337455]
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))

Best parameters: {'subsample_for_bin': 257455, 'num_leaves': 11, 'n_estimators': 269, 'max_depth': 25, 'learning_rate': 0.2, 'boosting_type': 'dart'}
              precision    recall  f1-score   support

           0       0.81      0.70      0.75       336
           1       0.70      0.81      0.75       288

    accuracy                           0.75       624
   macro avg       0.75      0.75      0.75       624
weighted avg       0.76      0.75      0.75       624



## Emotion

In [54]:
df_test_scaled_e = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [55]:
df_train_scaled_e = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [56]:
### One-hot encoding

#### Train 

df_train_scaled_e['emotion'] = pd.factorize(df_train_scaled_e['emotion'])[0]
df_train_e = pd.get_dummies(df_train_scaled_e)

#### Test

df_test_scaled_e['emotion'] = pd.factorize(df_test_scaled_e['emotion'])[0]
df_test_e = pd.get_dummies(df_test_scaled_e)

In [57]:
X_train = df_train_e.drop(['emotion'], axis=1)
y_train = df_train_e['emotion']
X_test = df_test_e.drop(['emotion'], axis=1)
y_test = df_test_e['emotion']

In [42]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.3906939937054613, 'max_depth': 43, 'n_estimators': 206, 'num_leaves': 19, 'subsample_for_bin': 261228}
              precision    recall  f1-score   support

           0       0.47      0.42      0.44        48
           1       0.57      0.65      0.60        96
           2       0.51      0.36      0.42        96
           3       0.39      0.36      0.38        96
           4       0.54      0.78      0.64        96
           5       0.56      0.36      0.44        96
           6       0.42      0.44      0.43        48
           7       0.35      0.46      0.40        48

    accuracy                           0.49       624
   macro avg       0.48      0.48      0.47       624
weighted avg       0.49      0.49      0.48       624



In [58]:
param_distribs = {
    'boosting_type': ['gbdt'],
    'n_estimators': [176,186,196,206,216,226,236,246,256,266],
    'max_depth': [35,36,37,38,39,340,41,42,43,44,45,46,47],
    'learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'num_leaves': [14,15,16,17,18,19,20,21,22,23,24,25],
    "subsample_for_bin" : [221228,231228,241228,251228,261228,271228,272228,282228,292228]
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))

Best parameters: {'subsample_for_bin': 251228, 'num_leaves': 18, 'n_estimators': 256, 'max_depth': 36, 'learning_rate': 0.2, 'boosting_type': 'gbdt'}
              precision    recall  f1-score   support

           0       0.49      0.40      0.44        48
           1       0.54      0.70      0.61        96
           2       0.51      0.38      0.43        96
           3       0.42      0.30      0.35        96
           4       0.53      0.79      0.64        96
           5       0.58      0.33      0.42        96
           6       0.42      0.50      0.46        48
           7       0.36      0.50      0.42        48

    accuracy                           0.49       624
   macro avg       0.48      0.49      0.47       624
weighted avg       0.50      0.49      0.48       624



# Utilizzo della conversione automatica del modello per le categoriche

## Variabile target --> emotional_intensity

LGBMClassifier() utilizza per la conversione delle categoriche il metodo Fisher, che almeno teoricamente, dovrebbe performare meglio della one-hot encoding 
https://lightgbm.readthedocs.io/en/latest/Advanced-Topics.html

L'unica azione di preprocessing con è la standardizzazione

In [48]:
df_test_scaled_ei = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [49]:
df_train_scaled_ei = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [50]:
# Qua trasformo le categoriche (che normalmente assumo il datatype "object") nel datatype "category". L'operazione viene svolta 
# sia per il train che per il test set, escludendo la variabile target.
cat_cols = ['vocal_channel', 'emotion', 'statement', 'repetition', 'sex']

for col in cat_cols:
    df_train_scaled_s[col] = df_train_scaled_s[col].astype('category')
    df_test_scaled_s[col] = df_test_scaled_s[col].astype('category')


In [51]:
X_train = df_train_scaled_ei.drop(['emotional_intensity'], axis=1)
y_train = df_train_scaled_ei['emotional_intensity']
X_test = df_test_scaled_ei.drop(['emotional_intensity'], axis=1)
y_test = df_test_scaled_ei['emotional_intensity']

In [52]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.013257113655901081, 'max_depth': 15, 'n_estimators': 282, 'num_leaves': 7, 'subsample_for_bin': 258053}
              precision    recall  f1-score   support

      normal       0.80      0.71      0.76       336
      strong       0.70      0.80      0.75       288

    accuracy                           0.75       624
   macro avg       0.75      0.75      0.75       624
weighted avg       0.76      0.75      0.75       624



## Variabile target --> Sex

In [59]:
df_test_scaled_s = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [60]:
df_train_scaled_s = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [61]:
# Qua trasformo le categoriche (che normalmente assumo il datatype "object") nel datatype "category". L'operazione viene svolta 
# sia per il train che per il test set, escludendo la variabile target.
cat_cols = ['vocal_channel', 'emotion', 'statement', 'repetition', 'emotional_intensity']

for col in cat_cols:
    df_train_scaled_s[col] = df_train_scaled_s[col].astype('category')
    df_test_scaled_s[col] = df_test_scaled_s[col].astype('category')



In [62]:
X_train = df_train_scaled_s.drop(['sex'], axis=1)
y_train = df_train_scaled_s['sex']
X_test = df_test_scaled_s.drop(['sex'], axis=1)
y_test = df_test_scaled_s['sex']

In [63]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'dart', 'learning_rate': 0.22229964825261933, 'max_depth': 6, 'n_estimators': 359, 'num_leaves': 33, 'subsample_for_bin': 245758}
              precision    recall  f1-score   support

           F       0.99      0.77      0.86       312
           M       0.81      0.99      0.89       312

    accuracy                           0.88       624
   macro avg       0.90      0.88      0.88       624
weighted avg       0.90      0.88      0.88       624



## Variabile target --> Vocal_channel

In [64]:
df_test_scaled_v = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [65]:
df_train_scaled_v = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [66]:
# Qua trasformo le categoriche (che normalmente assumo il datatype "object") nel datatype "category". L'operazione viene svolta 
# sia per il train che per il test set, escludendo la variabile target.
cat_cols = ['sex', 'emotion', 'statement', 'repetition', 'emotional_intensity']

for col in cat_cols:
    df_train_scaled_v[col] = df_train_scaled_v[col].astype('category')
    df_test_scaled_v[col] = df_test_scaled_v[col].astype('category')


In [67]:
X_train = df_train_scaled_v.drop(['vocal_channel'], axis=1)
y_train = df_train_scaled_v['vocal_channel']
X_test = df_test_scaled_v.drop(['vocal_channel'], axis=1)
y_test = df_test_scaled_v['vocal_channel']

In [68]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=75, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'dart', 'learning_rate': 0.2682695795279725, 'max_depth': 39, 'n_estimators': 313, 'num_leaves': 48, 'subsample_for_bin': 248747}
              precision    recall  f1-score   support

        song       0.95      0.99      0.97       264
      speech       0.99      0.96      0.97       360

    accuracy                           0.97       624
   macro avg       0.97      0.97      0.97       624
weighted avg       0.97      0.97      0.97       624



## Variabile target --> Emotion

In [84]:
df_test_scaled_e = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [85]:
df_train_scaled_e = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

In [71]:
# Qua trasformo le categoriche (che normalmente assumo il datatype "object") nel datatype "category". L'operazione viene svolta 
# sia per il train che per il test set, escludendo la variabile target.
cat_cols = ['sex', 'vocal_channel', 'statement', 'repetition', 'emotional_intensity']

for col in cat_cols:
    df_train_scaled_e[col] = df_train_scaled_e[col].astype('category')
    df_test_scaled_e[col] = df_test_scaled_e[col].astype('category')


In [72]:
X_train = df_train_scaled_e.drop(['emotion'], axis=1)
y_train = df_train_scaled_e['emotion']
X_test = df_test_scaled_e.drop(['emotion'], axis=1)
y_test = df_test_scaled_e['emotion']

In [73]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart','goss'],
    'n_estimators': randint(low=50, high=500),
    'max_depth': randint(low=3, high=50),
    'learning_rate': np.logspace(-4, 0, 50),
    'num_leaves': randint(low=5, high=50),
    "subsample_for_bin" : randint(low=200000, high=300000) 
}

clf = LGBMClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.3906939937054613, 'max_depth': 43, 'n_estimators': 206, 'num_leaves': 19, 'subsample_for_bin': 261228}
              precision    recall  f1-score   support

       angry       0.54      0.78      0.64        96
        calm       0.57      0.65      0.60        96
     disgust       0.42      0.44      0.43        48
     fearful       0.56      0.36      0.44        96
       happy       0.51      0.36      0.42        96
     neutral       0.47      0.42      0.44        48
         sad       0.39      0.36      0.38        96
   surprised       0.35      0.46      0.40        48

    accuracy                           0.49       624
   macro avg       0.48      0.48      0.47       624
weighted avg       0.49      0.49      0.48       624



Si nota una peggioramento dei modelli utilizzando la trasformazione delle categoriali integrata nel modello, anche se secondo la teoria (stesso link riportato sopra)dovrebbe essere migliore. 


# HistGradientBoostingClassifier ()

Il prof aveva sottolineato come per questo classificatore non fosse neanche necessario fare la grid perchè avrebbe performato bene

## target --> sex

In [ ]:
### One-hot encoding

#### Train 

df_train_scaled_s['sex'] = pd.factorize(df_train_scaled_s['sex'])[0]
df_train_s = pd.get_dummies(df_train_scaled_s)

#### Test

df_test_scaled_s['sex'] = pd.factorize(df_test_scaled_s['sex'])[0]
df_test_s = pd.get_dummies(df_test_scaled_s)

In [25]:
X_train = df_train_s.drop(['sex'], axis=1)
y_train = df_train_s['sex']
X_test = df_test_s.drop(['sex'], axis=1)
y_test = df_test_s['sex']

In [33]:
param_distribs = {
    "loss" : ["binary_crossentropy"],
    "learning_rate": np.logspace(-4, 0, 50),
    "max_depth": randint(low=3, high=50),
    "max_iter": randint(low=50, high=500),
    "max_leaf_nodes": randint(low=10, high=50),
    "l2_regularization": np.logspace(-4, 0, 50)
}

clf = HistGradientBoostingClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'l2_regularization': 0.12648552168552957, 'learning_rate': 0.019306977288832496, 'loss': 'binary_crossentropy', 'max_depth': 17, 'max_iter': 156, 'max_leaf_nodes': 17}
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       312
           1       0.99      0.77      0.87       312

    accuracy                           0.88       624
   macro avg       0.90      0.88      0.88       624
weighted avg       0.90      0.88      0.88       624



## Target --> vocal channel

In [75]:
### One-hot encoding

#### Train 

df_train_scaled_v['vocal_channel'] = pd.factorize(df_train_scaled_v['vocal_channel'])[0]
df_train_v = pd.get_dummies(df_train_scaled_v)

#### Test

df_test_scaled_v['vocal_channel'] = pd.factorize(df_test_scaled_v['vocal_channel'])[0]
df_test_v = pd.get_dummies(df_test_scaled_v)

In [76]:
X_train = df_train_v.drop(['vocal_channel'], axis=1)
y_train = df_train_v['vocal_channel']
X_test = df_test_v.drop(['vocal_channel'], axis=1)
y_test = df_test_v['vocal_channel']

In [77]:
param_distribs = {
    "loss" : ["binary_crossentropy"],
    "learning_rate": np.logspace(-4, 0, 50),
    "max_depth": randint(low=3, high=50),
    "max_iter": randint(low=50, high=500),
    "max_leaf_nodes": randint(low=10, high=50),
    "l2_regularization": np.logspace(-4, 0, 50)
}

clf = HistGradientBoostingClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'l2_regularization': 0.004291934260128779, 'learning_rate': 0.12648552168552957, 'loss': 'binary_crossentropy', 'max_depth': 21, 'max_iter': 264, 'max_leaf_nodes': 20}
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       360
           1       0.94      0.99      0.96       264

    accuracy                           0.97       624
   macro avg       0.96      0.97      0.97       624
weighted avg       0.97      0.97      0.97       624



## Target --> emotional_intensity

In [81]:
### One-hot encoding

#### Train 

df_train_scaled_ei['emotional_intensity'] = pd.factorize(df_train_scaled_ei['emotional_intensity'])[0]
df_train_ei = pd.get_dummies(df_train_scaled_ei)

#### Test

df_test_scaled_ei['emotional_intensity'] = pd.factorize(df_test_scaled_ei['emotional_intensity'])[0]
df_test_ei = pd.get_dummies(df_test_scaled_ei)

In [82]:
X_train = df_train_ei.drop(['emotional_intensity'], axis=1)
y_train = df_train_ei['emotional_intensity']
X_test = df_test_ei.drop(['emotional_intensity'], axis=1)
y_test = df_test_ei['emotional_intensity']

In [83]:
param_distribs = {
    "loss" : ["binary_crossentropy"],
    "learning_rate": np.logspace(-4, 0, 50),
    "max_depth": randint(low=3, high=50),
    "max_iter": randint(low=50, high=500),
    "max_leaf_nodes": randint(low=10, high=50),
    "l2_regularization": np.logspace(-4, 0, 50)
}

clf = HistGradientBoostingClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'l2_regularization': 0.004291934260128779, 'learning_rate': 0.12648552168552957, 'loss': 'binary_crossentropy', 'max_depth': 21, 'max_iter': 264, 'max_leaf_nodes': 20}
              precision    recall  f1-score   support

           0       0.77      0.78      0.77       336
           1       0.74      0.73      0.73       288

    accuracy                           0.75       624
   macro avg       0.75      0.75      0.75       624
weighted avg       0.75      0.75      0.75       624



## target --> emotion

In [86]:
### One-hot encoding

#### Train 

df_train_scaled_e['emotion'] = pd.factorize(df_train_scaled_e['emotion'])[0]
df_train_e = pd.get_dummies(df_train_scaled_e)

#### Test

df_test_scaled_e['emotion'] = pd.factorize(df_test_scaled_e['emotion'])[0]
df_test_e = pd.get_dummies(df_test_scaled_e)

In [87]:
X_train = df_train_e.drop(['emotion'], axis=1)
y_train = df_train_e['emotion']
X_test = df_test_e.drop(['emotion'], axis=1)
y_test = df_test_e['emotion']

In [89]:
param_distribs = {
    "loss" : ["categorical_crossentropy"],  # categorical_crossentropy per una multiclass classification
    "learning_rate": np.logspace(-4, 0, 50),
    "max_depth": randint(low=3, high=50),
    "max_iter": randint(low=50, high=500),
    "max_leaf_nodes": randint(low=10, high=50),
    "l2_regularization": np.logspace(-4, 0, 50)
}

clf = HistGradientBoostingClassifier()

random_search = RandomizedSearchCV(clf, param_distributions=param_distribs, n_iter=50, cv=5, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_clf = random_search.best_estimator_ 
best_clf.fit(X_train, y_train)

y_pred = best_clf.predict(X_test)

print("Best parameters:", random_search.best_params_)
print(classification_report(y_test, y_pred))


Best parameters: {'l2_regularization': 0.0002559547922699536, 'learning_rate': 0.22229964825261933, 'loss': 'categorical_crossentropy', 'max_depth': 6, 'max_iter': 359, 'max_leaf_nodes': 38}
              precision    recall  f1-score   support

           0       0.54      0.31      0.39        48
           1       0.52      0.68      0.59        96
           2       0.47      0.40      0.43        96
           3       0.41      0.33      0.37        96
           4       0.48      0.76      0.59        96
           5       0.59      0.39      0.47        96
           6       0.54      0.42      0.47        48
           7       0.35      0.46      0.40        48

    accuracy                           0.48       624
   macro avg       0.49      0.47      0.46       624
weighted avg       0.49      0.48      0.47       624



# note

Ho provato anche ad utilizzare la XGBoost, ho avuto difficoltà perchè, pur stringendo al massimo i parametri, il classificatore non gira